# **Laboratorio: Explorando la maleabilidad en AES-CBC**

## **Importación de Paquetes y Librerías**

In [2]:
import os
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad

## **Funciones Auxialiares**

### **Funciones de Cifrado y Descifrado**

In [14]:
def encrypt(mode, key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # Nombre del archivo de salida cifrado
        outputFile = filename+".enc"
        # Obtener el tamaño del archivo original y rellenar con ceros a la izquierda para que tenga 16 bytes
        filesize = str(os.path.getsize(filename)).zfill(16)
        # Generar un vector de inicialización (IV) aleatorio de 16 bytes

        # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
        if mode == "ECB":
            encryptor= AES.new(key, AES.MODE_ECB)
        elif mode == "CBC":
            IV = Random.new().read(16)
            encryptor= AES.new(key, AES.MODE_CBC, IV)
        else:
            print("Error de mode")
            return

        # Abrir el archivo de entrada en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Escribir el tamaño del archivo original codificado en utf-8
                        outfile.write(filesize.encode('utf-8'))


                        # Escribir el vector de inicialización (IV)
                        if mode == "CBC":
                            outfile.write(IV)


                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break
                                # Si el tamaño del chunk no es múltiplo de 16, rellenar con bytes nulos
                                elif len(chunk)%16 != 0:
                                        chunk=pad(chunk, 16)

                                # Cifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(encryptor.encrypt(chunk))

# Función para descifrar un archivo
def decrypt(mode, key, filename, output_filename=None):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # ruta del archivo de salida descifrado (eliminar la extensión .enc)
        outputFile = filename[:-4] if output_filename is None else output_filename

        # Abrir el archivo de entrada cifrado en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Leer el tamaño del archivo original (los primeros 16 bytes) y convertirlo a entero
                filesize = int(infile.read(16))

                # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
                if mode == "ECB":
                    decryptor= AES.new(key, AES.MODE_ECB)
                elif mode == "CBC":
                    IV = infile.read(16)
                    decryptor= AES.new(key, AES.MODE_CBC, IV)
                else:
                    print("Error de mode")
                    return

                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break

                                # Descifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(decryptor.decrypt(chunk))

                        # Truncar el archivo de salida al tamaño original
                        outfile.truncate(filesize)

### **Funciones Comparadoras de Archivos**
#### **Comparador de Archivos por Bloques**

In [4]:
def compare_files_per_block(file1, file2, block_size=16):
    """
    Compara dos archivos y reporta únicamente los bloques que están corrompidos.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()

    corrupted_blocks = set()

    # Comparar byte por byte y registrar bloques corrompidos
    for i in range(min(len(data1), len(data2))):
        if data1[i] != data2[i]:
            block_index = i // block_size + 1
            corrupted_blocks.add(block_index)

    # Mostrar resultados
    if corrupted_blocks:
        sorted_blocks = sorted(corrupted_blocks)
        print(f"Bloques corrompidos: {sorted_blocks}")
        print(f"Total de bloques afectados: {len(corrupted_blocks)}")
    else:
        print("Los archivos son idénticos - no hay bloques corrompidos")

#### **Comparador de Archivos por Bytes**

In [5]:
def compare_files_per_byte(file1, file2, block_size=16, max_diffs=64):
    """
    Compara dos archivos byte por byte y muestra diferencias.
    Agrupa por bloques para ver qué zonas se dañaron.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()

    for i, (b1, b2) in enumerate(zip(data1, data2)):
        if b1 != b2:
            block_index = i // block_size
            print(f"Diferencia en byte {i} (bloque {block_index}): {b1} != {b2}")
            max_diffs -= 1
            if max_diffs == 0:
                print("...más diferencias ocultas...")
                break

### **Función Generadora de Llave**

In [24]:
# Función para generar una clave a partir de una contraseña usando SHA256
def getKey(password):
    # Crear un objeto hash SHA256
    hasher = SHA256.new(password.encode("utf-8"))
    # Devolver el resumen del hash (la clave)
    return hasher.digest()

## **Parte 1: Prueba Básica**

1. Se elige el archivo `test.pdf` (mayor a 1 KB).
2. Se cifra usando AES-CBC.
3. Se descifra y se verifica que el archivo recuperado coincide con el original.

In [23]:
encrypt("CBC", SHA256.new(b"password").digest(), "test.pdf")

decrypt("CBC", SHA256.new(b"password").digest(), "test.pdf.enc", "Resultados/CBC - Basic Test/decrypted_test.pdf")

# Comparar archivos por bloques
compare_files_per_block("test.pdf", "Resultados/CBC - Basic Test/decrypted_test.pdf")

# Comparar archivos por bytes
compare_files_per_byte("test.pdf", "Resultados/CBC - Basic Test/decrypted_test.pdf")


Los archivos son idénticos - no hay bloques corrompidos


Los resultados de la comparación muestran que el archivo original (`test.pdf`) y el archivo recuperado (`decrypted_test.pdf`) son idénticos. Esto confirma que el proceso de cifrado y descifrado con AES-CBC se realizó correctamente, preservando la integridad del archivo. Por lo tanto, se puede confiar en que el esquema implementado recupera el archivo original sin pérdida ni alteración de datos.

## **Parte 2: Flipping de Bytes (CBC)**
1. Se define la función `bit_flipper()`, la cual altera un bit dentro del tercer bloque de ciphertext.
1. Se modifica el archivo cifrado *test.pdf.enc* con la función `bit_flipper()`.
2. Se descifra el archivo modificado con la misma contraseña.
3. Se comparan el archivo orginal con el archivo modificado descifrado.

In [25]:
# Función para alterar un bit en el tercer bloque de ciphertext
def bit_flipper(file_input, file_output):
    with open(file_input, "rb") as f:
        data = bytearray(f.read())
        data[50] ^= 0x01

    # cambia un bit dentro de C_3 ( ciphertext )
    with open(file_output, "wb") as f:
        f.write(data)


# Clave para cifrado y descifrado
key = SHA256.new(b"password").digest()

# Cifrar el archivo original
encrypt("CBC", key, "test.pdf")

# Flipping bytes
bit_flipper("test.pdf.enc", "Resultados/CBC - Bit Flip/test_flipped.pdf.enc")

# Descifrar el archivo con el bit alterado
decrypt("CBC", key, "Resultados/CBC - Bit Flip/test_flipped.pdf.enc", "Resultados/CBC - Bit Flip/decrypted_flipped_test.pdf")

# Comparar archivos por bloques
compare_files_per_block("test.pdf", "Resultados/CBC - Bit Flip/decrypted_flipped_test.pdf")




Bloques corrompidos: [2, 3]
Total de bloques afectados: 2


Al realizar la prueba de flipping de bytes en el modo CBC, se alteró un bit dentro del tercer bloque del ciphertext. Tras descifrar el archivo modificado y comparar con el original, se observó que dos bloques resultaron corrompidos: el bloque alterado y el bloque anterior. Este resultado evidencia la propiedad de propagación de errores en CBC, donde una modificación en el ciphertext afecta no solo el bloque correspondiente, sino también el bloque previo al descifrar. Así, se demuestra la maleabilidad y el impacto de los cambios en la estructura de los bloques cifrados bajo este modo.

### **¿Por qué se corrompen dos bloques consecutivos en CBC?**

En el modo **Cipher Block Chaining (CBC)**, el descifrado de cada bloque se realiza con la fórmula:

\[
P_i = D_{AES}(C_i) \oplus C_{i-1}, \quad i \in \{2, \dots, n\}
\]

donde:  
- \(P_i\): bloque de **plaintext** recuperado.  
- \(C_i\): bloque de **ciphertext** recibido.  
- \(D_{AES}\): función de descifrado del algoritmo AES.  
- \(C_{i-1}\): bloque de **ciphertext** anterior.  

---

#### Caso de la alteración de un bit en \(C_3\):

1. **Bloque \(P_3\):**  
   - El descifrado depende directamente de \(C_3\).  
   - Como se alteró un bit en \(C_3\), al aplicar \(D_{AES}(C_3)\) el resultado será un valor completamente distinto.  
   - Por lo tanto, **todo el bloque \(P_3\) aparece corrompido**.  

2. **Bloque \(P_4\):**  
   - El descifrado de \(P_4\) se hace como:  
     \[
     P_4 = D_{AES}(C_4) \oplus C_3
     \]  
   - Aunque \(C_4\) esté intacto, el valor de \(C_3\) ya está modificado.  
   - Esto provoca que al hacer la operación XOR, **el bloque \(P_4\) también se corrompa**.  

3. **Bloques posteriores (\(P_5, P_6, \dots\)):**  
   - Se descifran como:  
     \[
     P_i = D_{AES}(C_i) \oplus C_{i-1}
     \]  
   - Aquí el \(C_{i-1}\) vuelve a ser válido (no alterado).  
   - Por eso, **el error no se propaga más allá de un bloque adicional**.  

 ## **Parte 3: Intercambio de Bloques**

1. Se define la función `swap_blocks()`, la cual intercambia dos bloques del archivo cifrado.
2. Se cifra el archivo original.
1. Se modifica el archivo cifrado *test.pdf.enc* con la función `swap_blocks()`.
2. Se descifra el archivo modificado con la misma contraseña.
3. Se comparan el archivo orginal con el archivo modificado descifrado.

In [26]:
# Función para intercambiar bloques en un archivo cifrado
def swap_blocks(input_file, output_file, block_size=16, i=2, j=3):
    """
    Intercambia los bloques Ci y Cj de un archivo cifrado en formato:
    C0 || C1 || C2 ... Cn
    block_size normalmente = 16 (AES).
    """

    with open(input_file, "rb") as f:
        data = bytearray(f.read())

    # print(data)
    # Calcular offset en bytes de cada bloque
    start_i, end_i = i * block_size, (i + 1) * block_size
    start_j, end_j = j * block_size, (j + 1) * block_size

    # Intercambiar bloques
    block_i = data[start_i:end_i]
    block_j = data[start_j:end_j]
    data[start_i:end_i], data[start_j:end_j] = block_j, block_i

    with open(output_file, "wb") as f:
        f.write(data)

    print(f"Intercambiados C{i} y C{j} en {input_file}, guardado en {output_file}")

# Encriptar archivo
encrypt("CBC", getKey("1234"), "test.pdf")

# Cambio de bloques
swap_blocks("test.pdf.enc", "Resultados/CBC - Block Swap/test_swap.pdf.enc", 16, 3, 5)

# Descifrado del archivo modificado
decrypt("CBC", getKey("1234"), "Resultados/CBC - Block Swap/test_swap.pdf.enc")

# Comparación de Archivos
compare_files_per_byte("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")
print(
    "====================================================================================================================="
)
compare_files_per_block("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")

Intercambiados C3 y C5 en test.pdf.enc, guardado en Resultados/CBC - Block Swap/test_swap.pdf.enc
Diferencia en byte 16 (bloque 1): 32 != 89
Diferencia en byte 17 (bloque 1): 48 != 118
Diferencia en byte 18 (bloque 1): 32 != 147
Diferencia en byte 19 (bloque 1): 111 != 210
Diferencia en byte 20 (bloque 1): 98 != 22
Diferencia en byte 21 (bloque 1): 106 != 3
Diferencia en byte 22 (bloque 1): 10 != 216
Diferencia en byte 23 (bloque 1): 60 != 240
Diferencia en byte 24 (bloque 1): 60 != 203
Diferencia en byte 25 (bloque 1): 10 != 48
Diferencia en byte 26 (bloque 1): 47 != 184
Diferencia en byte 27 (bloque 1): 76 != 161
Diferencia en byte 28 (bloque 1): 101 != 125
Diferencia en byte 29 (bloque 1): 110 != 16
Diferencia en byte 30 (bloque 1): 103 != 183
Diferencia en byte 31 (bloque 1): 116 != 129
Diferencia en byte 32 (bloque 2): 104 != 215
Diferencia en byte 33 (bloque 2): 32 != 179
Diferencia en byte 34 (bloque 2): 49 != 58
Diferencia en byte 35 (bloque 2): 48 != 36
Diferencia en byte 36 (

## **Parte 3 bis: Implementación con ECB**
### **Bit Flipping (ECB)**

In [27]:
encrypt("ECB", getKey("1234"), "test.pdf")
bit_flipper("test.pdf.enc", "Resultados/ECB - Bit flip/test_flip.pdf.enc")
decrypt("ECB", getKey("1234"), "Resultados/ECB - Bit flip/test_flip.pdf.enc")

compare_files_per_byte("test.pdf", "Resultados/ECB - Bit flip/test_flip.pdf")
print(
    "====================================================================================================================="
)
compare_files_per_block("test.pdf", "Resultados/ECB - Bit flip/test_flip.pdf")

Diferencia en byte 32 (bloque 2): 104 != 168
Diferencia en byte 33 (bloque 2): 32 != 103
Diferencia en byte 34 (bloque 2): 49 != 8
Diferencia en byte 35 (bloque 2): 48 != 55
Diferencia en byte 36 (bloque 2): 49 != 143
Diferencia en byte 37 (bloque 2): 51 != 0
Diferencia en byte 38 (bloque 2): 32 != 112
Diferencia en byte 39 (bloque 2): 32 != 5
Diferencia en byte 40 (bloque 2): 32 != 90
Diferencia en byte 41 (bloque 2): 32 != 95
Diferencia en byte 42 (bloque 2): 32 != 30
Diferencia en byte 43 (bloque 2): 32 != 235
Diferencia en byte 44 (bloque 2): 10 != 188
Diferencia en byte 45 (bloque 2): 47 != 154
Diferencia en byte 46 (bloque 2): 70 != 114
Diferencia en byte 47 (bloque 2): 105 != 100
Bloques corrompidos: [3]
Total de bloques afectados: 1


### **Intercambio de Bloques (ECB)**

In [28]:
encrypt("ECB", getKey("1234"), "test.pdf")
swap_blocks("test.pdf.enc", "Resultados/ECB - Block Swap/test_swap.pdf.enc", 16, 3, 5)
decrypt("ECB", getKey("1234"), "Resultados/ECB - Block Swap/test_swap.pdf.enc")
compare_files_per_byte("test.pdf", "Resultados/ECB - Block Swap/test_swap.pdf")
print(
    "====================================================================================================================="
)
compare_files_per_block("test.pdf", "Resultados/ECB - Block Swap/test_swap.pdf")

Intercambiados C3 y C5 en test.pdf.enc, guardado en Resultados/ECB - Block Swap/test_swap.pdf.enc
Diferencia en byte 32 (bloque 2): 104 != 101
Diferencia en byte 33 (bloque 2): 32 != 10
Diferencia en byte 34 (bloque 2): 49 != 62
Diferencia en byte 35 (bloque 2): 48 != 62
Diferencia en byte 36 (bloque 2): 49 != 10
Diferencia en byte 37 (bloque 2): 51 != 115
Diferencia en byte 38 (bloque 2): 32 != 116
Diferencia en byte 39 (bloque 2): 32 != 114
Diferencia en byte 40 (bloque 2): 32 != 101
Diferencia en byte 41 (bloque 2): 32 != 97
Diferencia en byte 42 (bloque 2): 32 != 109
Diferencia en byte 43 (bloque 2): 32 != 10
Diferencia en byte 44 (bloque 2): 10 != 120
Diferencia en byte 45 (bloque 2): 47 != 218
Diferencia en byte 46 (bloque 2): 70 != 189
Diferencia en byte 47 (bloque 2): 105 != 87
Diferencia en byte 64 (bloque 4): 101 != 104
Diferencia en byte 65 (bloque 4): 10 != 32
Diferencia en byte 66 (bloque 4): 62 != 49
Diferencia en byte 67 (bloque 4): 62 != 48
Diferencia en byte 68 (bloque

## **Parte 4: Reflexión**
**Confidencialidad:**  
AES-CBC y AES-ECB protegen la confidencialidad del mensaje, ya que sin la clave correcta no es posible recuperar el contenido original. Sin embargo, AES-ECB presenta una debilidad importante: bloques idénticos de texto plano generan bloques idénticos de texto cifrado, lo que permite detectar patrones y puede revelar información sobre la estructura del mensaje. En cambio, AES-CBC utiliza un vector de inicialización (IV) y encadenamiento de bloques, lo que elimina estos patrones y mejora la confidencialidad.

**Integridad:**  
Ninguno de los modos de operación (CBC o ECB) garantiza la integridad del archivo por sí solo. Un atacante puede modificar el archivo cifrado (por ejemplo, alterando o intercambiando bloques) y, aunque el contenido descifrado será incorrecto, el sistema no detectará la manipulación. Para proteger la integridad, es necesario añadir un código de autenticación de mensaje (MAC) o emplear un modo autenticado como AES-GCM (AEAD), que detecta cualquier alteración en el ciphertext.

**Maleabilidad y ataques:**  
AES-CBC y AES-ECB son modos maleables: un atacante puede modificar el ciphertext y provocar cambios predecibles en el texto plano descifrado. En CBC, un bit flip en el ciphertext afecta el bloque correspondiente y el siguiente, lo que puede ser aprovechado para alterar partes específicas del archivo, como cabeceras o metadatos. En ECB, la maleabilidad es aún mayor, ya que los bloques pueden ser intercambiados o modificados de forma independiente, permitiendo manipular el mensaje sin restricciones. Esta maleabilidad representa un riesgo práctico en sistemas de almacenamiento de archivos cifrados, ya que un atacante podría modificar datos críticos sin ser detectado, comprometiendo la integridad y funcionalidad de los archivos. Por ello, es fundamental complementar el cifrado con mecanismos de autenticación.